# Auto XRD analyisis

In this notebook we exemplify how to train an [XRD-AutoAnalyzer](https://github.com/njszym/XRD-AutoAnalyzer) (CNN) model on a chemical space. For that, we will use the nomad-auto-xrd plugins that has come handy functions to train the model and to save it in NOMAD.

Then, we will save the model(s) trainned as an entry in NOMAD, so we can serach for them and reuse them easily. 
Once we have done this, we will analyse some of the diffraction patterns that we have already uploaded in NOMAD, to match the phases to the diffraction patterns. 

### (!!) Some temporary installers until this gets solved in NORTH

```python
!pip install nomad_auto_xrd@git+https://github.com/FAIRmat-NFDI/nomad-auto-xrd
!pip install --upgrade pyxtal
```

First let's run some imports

In [12]:
import warnings

# Suppress specific warnings
warnings.filterwarnings('ignore', category=UserWarning, module='pymatgen.io.cif')
warnings.filterwarnings('ignore', category=DeprecationWarning, module='spglib')

In [ ]:
from nomad_auto_xrd.train_xrd_cnn import ModelConfig, run_xrd_model

The `ModelConfig` class is defined in `train_xrd_cnn.py` and is used to define the model architecture and training parameters. The `run_xrd_model` function is used to train the model. The `run_xrd_model` function takes a `ModelConfig` object as an argument and returns a trained model. The trained model can then be used to make predictions on new XRD data.

Let's start by inspecting the default model configuration.

In [ ]:
config = ModelConfig()  # default config
config

## Train the model with the default configuration

You can simple run the model with the default configuration by running the following cell. For that you will need to have your cif files in the `all_cifs_dir` folder.

In [ ]:
# Run the model with the defaults
run_xrd_model(config)

To train the model, we need to provide a folder with the structure files that we want to use to train the model. We will store them in the `All_CIFs` folder, which is the default one used by [XRD-AutoAnalyzer](https://github.com/njszym/XRD-AutoAnalyzer). One can also update the `config`  to change the default parameters of the model. Here we will reduce the number of epochs to 1, to make the training faster. Also, we will set the `save_nomad_metadata` to `True` to create an archive of this model. This will allow us to find it and reuse later on to run inference on experimental data.

In [ ]:
# Redefine some model parameters
config = ModelConfig(
    num_epochs=1,
    save_nomad_metadata=True,
    skip_filter=True,
)

# Run the training
run_xrd_model(config)

## Tracking with Weights and Biases

Addionally, we wilcanl use [Weights and Biases](https://wandb.ai/site) to track the training of the model. This will allow us to monitor the training process and compare different models. To use Weights and Biases, you need to create an account and set up a project. After that, you will get an API key that you can use to authenticate your account.

You can then set the `wandb_project` and `wandb_entity` parameters in the `ModelConfig` object to the name of your project and you WandB user. We can track the link also in the NOMAD archive.

In [ ]:
import getpass

import wandb

# Prompt the user to input the API key securely
api_key = getpass.getpass('Enter your W&B API key: ')

# Login with the API key
wandb.login(key=api_key)

Do not forget to set the `wandd_entity` and `wandb_project` to your own user and project.

In [ ]:
# Redefine some model parameters
config = ModelConfig(
    num_epochs=1,
    save_nomad_metadata=True,
    enable_wandb=True,
    skip_filter=True,
    wandb_project='nomad_auto_xrd_test',
    wandb_entity='pepemarquez',
)

# Run the training
run_xrd_model(config)